In [1]:
#!/usr/bin/env python
# Configure Jupyter so figures appear in the notebook

%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

def make_system(beta, gamma, quarantine_rate):
    """Make a system object for the SIR model.

    beta: contact rate in days
    gamma: recovery rate in days

    returns: System object
    """
    init = State(S=89, I=1, Q =0, R=0)
    init /= sum(init)

    t0 = 0
    t_end = 7 * 14

    return System(init=init, t0=t0, t_end=t_end,
                  beta=beta, gamma=gamma, quarantine_rate=quarantine_rate)

def update_func(state, t, system):
    """Update the SIR model.

    state: State with variables S, I, R
    t: time step
    system: System with beta and gamma

    returns: State object
    """
    s, i, q, r = state

    infected = system.beta * i * s
    recovered = system.gamma * i
    quarantine_in = system.quarantine_rate * i
    quarantine_out = q * system.quarantine_rate
    # Quarantine rate is the percentage of infected people who are moved in/out quarantine at any given time


    s -= infected
    i += infected - recovered - quarantine_in
    q += quarantine_in - quarantine_out
    r += recovered + quarantine_out

    return State(S=s, I=i, R=r)

def run_simulation(system, update_func):
    """Runs a simulation of the system.

    system: System object
    update_func: function that updates state

    returns: TimeFrame
    """
    frame = TimeFrame(columns=system.init.index)
    frame.row[system.t0] = system.init

    for t in linrange(system.t0, system.t_end):
        frame.row[t+1] = update_func(frame.row[t], t, system)

    return frame

def calc_total_infected(results):
    """Fraction of population infected during the simulation.

    results: DataFrame with columns S, I, R

    returns: fraction of population
    """
    return get_first_value(results.S) - get_last_value(results.S)

beta = 0.333
gamma = 0.25
quarantine_rate = 0.2
system = make_system(beta, gamma, quarantine_rate)

results = run_simulation(system, update_func)
print(beta, gamma, calc_total_infected(results))


0.333 0.25 0.028825497819311652
